# Generate Query - LLM Powered Endpoint

## Import Necessary Libraries

In [30]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import RichPromptTemplate
import strawberry
from backend.db.example_nodes import EXAMPLE_NODES
from backend.core.config import settings

In [31]:
GRAPHQL_SCHEMA_STR = """input FilterInput {
  field: String!
  operation: FilterOperator!
  value: VALUE!
}

enum FilterOperator {
  EQ
  GT
  GTE
  LT
  LTE
  NEQ
}

input LogicalFilterInput {
  and_: [LogicalFilterInput!] = null
  or_: [LogicalFilterInput!] = null
  not_: LogicalFilterInput = null
  filter: FilterInput = null
}

type Mutation {
  createUser(name: String!, email: String!, profile: ProfileInput = null): User!
  updateUser(id: ID!, name: String = null, email: String = null, profile: ProfileInput = null): User!
  deleteUser(id: ID!): User!
  createProduct(name: String!, price: Float!): Product!
  updateProduct(id: ID!, name: String = null, price: Float = null): Product!
  deleteProduct(id: ID!): Product!
  createOrder(userId: String!, items: [OrderItemInput!]!, status: OrderStatus!): Order!
  updateOrder(id: ID!, userId: String = null, items: [OrderItemInput!] = null, status: OrderStatus = null): Order!
  deleteOrder(id: ID!): Order!
  createReview(productId: String!, userId: String!, rating: Int!, comment: String = null): Review!
}

type Order {
  id: ID!
  items: [OrderItem!]
  status: OrderStatus
  user: User
}

type OrderItem {
  quantity: Int
  product: Product
}

input OrderItemInput {
  productId: String!
  quantity: Int!
}

enum OrderStatus {
  PENDING
  ORDERED
  SHIPPED
  DELIVERED
  CANCELLED
}

type Product {
  id: ID!
  name: String
  price: Float
  reviews: [Review!]
}

type Profile {
  age: Int
  location: String
}

input ProfileInput {
  age: Int!
  location: String!
}

type Query {
  users(filters: LogicalFilterInput = null): [User!]!
  orders(filters: LogicalFilterInput = null): [Order!]!
  products(filters: LogicalFilterInput = null): [Product!]!
  reviews(filters: LogicalFilterInput = null): [Review!]!
  user(id: ID!): User!
  order(id: ID!): Order!
  product(id: ID!): Product!
}

type Review {
  id: ID!
  rating: Int
  comment: String
  createdAt: String
  product: Product
  user: User
}

type User {
  id: ID!
  name: String
  email: String
  profile: Profile
  orders: [Order!]
}
"""

## LLM Configuration

In [32]:
Settings.llm = OpenAI(model="gpt-4.1")

index = VectorStoreIndex(nodes=EXAMPLE_NODES)
retriever = index.as_retriever(similarity_top_k=2)

## Prompt Template 

In [ ]:
PROMPT_TEMPLATE_STR = """
You are a GraphQL expert. You are an assistant that converts natural language
queries into GraphQL queries. You must strictly follow the provided
GraphQL schema and generate queries based on it.

Here is the GraphQL schema you should use:
<schema>
{{ schema }}
</schema>

Here are some examples of how you should convert natural language to GraphQL:
<examples>
{{ examples }}
</examples>

Now it's your turn.

Query: {{ query_str }}
GraphQL:
"""


def get_examples_fn(**kwargs):
    """Retrieve relevant examples based on the input query."""
    query = kwargs["query_str"]
    examples = retriever.retrieve(query)
    return "\n\n".join(node.text for node in examples)


prompt_template = RichPromptTemplate(
    PROMPT_TEMPLATE_STR,
    function_mappings={"examples": get_examples_fn},
)

## Generate GraphQL Query

In [49]:
async def generate_graphql_query(
    natural_language_query: str, schema: str
) -> str:
    """Generate a GraphQL query using dynamic few-shot prompting."""
    prompt = prompt_template.format(
        query_str=natural_language_query,
        schema=schema,
    )
    # print(f"Prompt: \n {prompt}")
    llm = Settings.llm
    response = await llm.acomplete(prompt)
    return response.text.strip("```").strip("graphql")

## Validate and Execute GraphQL Query

In [50]:
async def validate_and_execute_query(
    graphql_query: str, schema: strawberry.Schema
):
    """Validate and execute the GraphQL query."""
    result = await schema.execute(graphql_query)
    if result.errors:
        raise ValueError(f"GraphQL execution errors: {result.errors}")
    return result.data

## Test the Generation of GraphQL query

In [52]:
user_query = input("Enter a query: ")
print(
    "Generated GraphQL query: \n"
    f"{await generate_graphql_query(user_query, GRAPHQL_SCHEMA_STR)}"
)

Generated GraphQL query: 
Please provide your natural language query so I can convert it into a GraphQL query according to the schema.
